In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from pandas.io import sql
from sqlalchemy import create_engine, inspect

## Get the data we want to analyze

In [2]:
files = ['/data/reddit1.csv', 
         '/data/reddit2.csv',
         '/data/reddit3.csv',
         '/data/reddit4.csv',
         '/data/reddit5.csv',
         '/data/reddit6.csv',
         '/data/reddit7.csv',
         '/data/reddit8.csv',
         '/data/reddit9.csv',
         '/data/reddit10.csv']

finalDf = pd.DataFrame()

for file in files:
    df = pd.read_csv("/data/reddit1.csv")
    df = df[['body', 'created_utc', 'controversiality']]
    #adding the controversial 
    controversial = df[df['controversiality'] == 1].copy().reset_index()
    del controversial['index']
    finalDf = finalDf.append(controversial,ignore_index = True)
    #adding the non-controversial 
    non_controversial = df[df['controversiality'] == 0].sample(frac=0.1, replace=False).copy().reset_index()
    del non_controversial['index']
    finalDf = finalDf.append(non_controversial, ignore_index = True)

df = finalDf

# Need to create columns for 
    1. Time of Day
    2. Day of Week
    3. Word Count
    4. top 500 word for controversial (tfidf value)
    5. top 500 word for non-controversial (tfidf value)

## Ignore

In [6]:
from nltk.tokenize import word_tokenize          
import operator

df['body'] = df['body'].astype('str')

comments = df['body']

word_count={}
punctuations = '''`!()-[]{};:'"\,<>./?@#$%^&*_~=+ '''

for comment in comments:
    for word in word_tokenize(comment.lower()):
        if word not in punctuations:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

sorted_x = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)